# 🚀 IPO Farming Backtest System

**Complete plug-and-play system for backtesting IPO day trading strategies**

## ✨ Features
- **No setup required** - Works out of the box
- **Dual mode** - Simulated data (always works) or IBKR live data (optional)
- **Smart fallbacks** - Continues even if IBKR connection fails
- **All-in-one** - Everything in this single notebook

## 📊 Just Run All Cells to Get:
- Window analysis of 91+ trading strategies
- Comprehensive visualizations
- Detailed performance reports
- Actionable trading insights

## 1️⃣ Setup & Installation

In [ ]:
# Install required packages (only runs if not already installed)
import subprocess
import sys
import importlib

packages = {
    'pandas': 'pandas',
    'numpy': 'numpy', 
    'matplotlib': 'matplotlib',
    'seaborn': 'seaborn',
    'openpyxl': 'openpyxl',
    'pytz': 'pytz',
    'tqdm': 'tqdm',
    'ib_insync': 'ib-insync'
}

for import_name, install_name in packages.items():
    try:
        importlib.import_module(import_name)
        print(f"✓ {import_name} already installed")
    except ImportError:
        print(f"Installing {install_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", install_name])
        print(f"✓ {install_name} installed")

print("\n✅ All packages ready!")

In [ ]:
# Import libraries
import os
import random
import warnings
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.gridspec import GridSpec
import pytz

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.float_format = '{:.2f}'.format

# Set random seed for reproducibility
np.random.seed(42)
random.seed(42)

# Create output directory
os.makedirs('outputs', exist_ok=True)

print("🎯 Environment ready!")
print(f"📁 Working directory: {os.getcwd()}")
print(f"📊 Output directory: {os.path.abspath('outputs')}")

## 2️⃣ Data Mode Selection

In [ ]:
# Configuration
DATA_MODE = 'SIMULATION'  # Options: 'SIMULATION' or 'IBKR'
USE_SAMPLE_TICKERS = True  # Use pre-selected tickers (faster) or analyze all from Excel
MAX_TICKERS = 20  # Number of tickers to analyze

# IBKR Settings (only used if DATA_MODE='IBKR')
IBKR_CONFIG = {
    'HOST': '127.0.0.1',
    'PORT': 7497,  # 7497 for Paper, 7496 for Live
    'CLIENT_ID': random.randint(1, 999),  # Random ID to avoid conflicts
    'REQUEST_DELAY': 5.0,
    'PACING_DELAY': 15.0
}

print(f"📊 Data Mode: {DATA_MODE}")
print(f"🎲 Using {'sample' if USE_SAMPLE_TICKERS else 'Excel'} tickers")
print(f"📈 Analyzing {MAX_TICKERS} tickers")

if DATA_MODE == 'IBKR':
    print(f"\n🔌 IBKR Settings:")
    print(f"   Host: {IBKR_CONFIG['HOST']}:{IBKR_CONFIG['PORT']}")
    print(f"   Client ID: {IBKR_CONFIG['CLIENT_ID']} (randomized to avoid conflicts)")

## 3️⃣ IPO Universe Setup

In [ ]:
# Pre-selected high-quality IPOs for quick testing
SAMPLE_IPO_UNIVERSE = [
    ('GOOGL', 'Alphabet Inc', '2004-08-19', 85.00, 'Tech'),
    ('META', 'Meta Platforms', '2012-05-18', 38.00, 'Tech'),
    ('NFLX', 'Netflix', '2002-05-23', 15.00, 'Media'),
    ('UBER', 'Uber', '2019-05-10', 45.00, 'Gig'),
    ('LYFT', 'Lyft', '2019-03-29', 72.00, 'Gig'),
    ('ABNB', 'Airbnb', '2020-12-10', 68.00, 'Travel'),
    ('DASH', 'DoorDash', '2020-12-09', 102.00, 'Delivery'),
    ('SNOW', 'Snowflake', '2020-09-16', 120.00, 'Cloud'),
    ('ZM', 'Zoom', '2019-04-18', 36.00, 'SaaS'),
    ('DOCU', 'DocuSign', '2018-04-27', 29.00, 'SaaS'),
    ('COIN', 'Coinbase', '2021-04-14', 250.00, 'Crypto'),
    ('HOOD', 'Robinhood', '2021-07-29', 38.00, 'Fintech'),
    ('PYPL', 'PayPal', '2015-07-06', 38.00, 'Fintech'),
    ('SQ', 'Block', '2015-11-19', 9.00, 'Fintech'),
    ('SHOP', 'Shopify', '2015-05-21', 17.00, 'Ecommerce'),
    ('ETSY', 'Etsy', '2015-04-16', 16.00, 'Ecommerce'),
    ('SPOT', 'Spotify', '2018-04-03', 165.90, 'Media'),
    ('ROKU', 'Roku', '2017-09-28', 14.00, 'Media'),
    ('MRNA', 'Moderna', '2018-12-07', 23.00, 'Biotech'),
    ('RIVN', 'Rivian', '2021-11-10', 78.00, 'EV'),
]

if USE_SAMPLE_TICKERS:
    # Use pre-selected tickers
    ipo_universe = pd.DataFrame(
        SAMPLE_IPO_UNIVERSE[:MAX_TICKERS],
        columns=['Ticker', 'Company', 'IPO_Date', 'IPO_Price', 'Sector']
    )
    ipo_universe['IPO_Date'] = pd.to_datetime(ipo_universe['IPO_Date'])
    print(f"✓ Using {len(ipo_universe)} pre-selected IPOs")
else:
    # Load from Excel file
    try:
        df = pd.read_excel('IPO-age.xlsx', sheet_name='1975-2024', dtype={'offer date': str})
        df['IPO_Date'] = pd.to_datetime(df['offer date'], format='%Y%m%d', errors='coerce')
        df = df[df['IPO_Date'] >= '2000-01-01']
        df = df.dropna(subset=['Ticker'])
        df['Ticker'] = df['Ticker'].astype(str).str.strip()
        df = df.rename(columns={'IPO name': 'Company'})
        
        # Take first N tickers
        ipo_universe = df[['Ticker', 'Company', 'IPO_Date']].head(MAX_TICKERS).copy()
        ipo_universe['IPO_Price'] = np.random.uniform(10, 100, len(ipo_universe))  # Simulated prices
        ipo_universe['Sector'] = 'Unknown'
        print(f"✓ Loaded {len(ipo_universe)} IPOs from Excel")
    except:
        print("⚠️ Could not load Excel file, using sample tickers instead")
        ipo_universe = pd.DataFrame(
            SAMPLE_IPO_UNIVERSE[:MAX_TICKERS],
            columns=['Ticker', 'Company', 'IPO_Date', 'IPO_Price', 'Sector']
        )
        ipo_universe['IPO_Date'] = pd.to_datetime(ipo_universe['IPO_Date'])

print(f"\n📊 IPO Universe Summary:")
print(f"   Date range: {ipo_universe['IPO_Date'].min().date()} to {ipo_universe['IPO_Date'].max().date()}")
print(f"   Sectors: {ipo_universe['Sector'].nunique()}")
print(f"\nFirst 5 IPOs:")
ipo_universe.head()

## 4️⃣ Data Generation/Collection

In [ ]:
def generate_ipo_day_data(ticker, ipo_date, ipo_price, sector='Tech'):
    """Generate realistic IPO day intraday data"""
    np.random.seed(hash(ticker) % 2**32)
    
    # Sector-specific patterns
    patterns = {
        'Tech': {'pop': 1.25, 'volatility': 0.005, 'trend': 0.0002},
        'Fintech': {'pop': 1.15, 'volatility': 0.004, 'trend': 0.0001},
        'Biotech': {'pop': 1.35, 'volatility': 0.008, 'trend': -0.0001},
        'EV': {'pop': 1.40, 'volatility': 0.009, 'trend': -0.0002},
        'SaaS': {'pop': 1.20, 'volatility': 0.004, 'trend': 0.0003},
        'Media': {'pop': 1.18, 'volatility': 0.005, 'trend': 0.0001},
        'Ecommerce': {'pop': 1.12, 'volatility': 0.003, 'trend': 0.0001},
        'Crypto': {'pop': 1.45, 'volatility': 0.010, 'trend': -0.0003},
    }
    
    pattern = patterns.get(sector, patterns['Tech'])
    
    # Generate time series (9:30 AM to 4:00 PM)
    eastern = pytz.timezone('America/New_York')
    start_time = pd.Timestamp(ipo_date).replace(hour=9, minute=30)
    end_time = pd.Timestamp(ipo_date).replace(hour=16, minute=0)
    times = pd.date_range(start_time, end_time, freq='1min')
    
    # Generate price path
    open_price = ipo_price * pattern['pop'] * np.random.uniform(0.95, 1.05)
    prices = [open_price]
    
    for i in range(1, len(times)):
        hour = times[i].hour
        
        # Intraday volatility patterns
        if hour < 10:  # Opening hour
            vol = pattern['volatility'] * 1.5
            trend = -0.0002
        elif hour < 12:  # Morning
            vol = pattern['volatility'] * 1.2
            trend = pattern['trend'] * 0.5
        elif hour < 14:  # Midday
            vol = pattern['volatility'] * 0.8
            trend = pattern['trend']
        else:  # Afternoon
            vol = pattern['volatility'] * 1.1
            trend = pattern['trend'] * 1.5
        
        change = np.random.normal(trend, vol)
        new_price = prices[-1] * (1 + change)
        prices.append(max(new_price, ipo_price * 0.5))  # Floor at 50% of IPO price
    
    # Create DataFrame
    df = pd.DataFrame({
        'datetime': times,
        'open': prices,
        'high': np.array(prices) * (1 + np.abs(np.random.normal(0, 0.002, len(prices)))),
        'low': np.array(prices) * (1 - np.abs(np.random.normal(0, 0.002, len(prices)))),
        'close': prices,
        'volume': np.random.gamma(2, 100000, len(prices)).astype(int)
    })
    
    # Ensure OHLC consistency
    df['high'] = df[['open', 'high', 'close']].max(axis=1)
    df['low'] = df[['open', 'low', 'close']].min(axis=1)
    
    return df

In [ ]:
# Data collection/generation based on mode
ipo_data = {}

if DATA_MODE == 'SIMULATION':
    print("🎲 Generating simulated IPO day data...\n")
    
    for _, row in ipo_universe.iterrows():
        ticker = row['Ticker']
        ipo_date = row['IPO_Date']
        ipo_price = row['IPO_Price']
        sector = row.get('Sector', 'Tech')
        
        # Generate data
        df = generate_ipo_day_data(ticker, ipo_date, ipo_price, sector)
        ipo_data[ticker] = df
        
        # Show summary
        open_price = df.iloc[0]['open']
        close_price = df.iloc[-1]['close']
        day_return = (close_price - open_price) / open_price * 100
        
        print(f"✓ {ticker:6s} | Open: ${open_price:6.2f} | Close: ${close_price:6.2f} | Return: {day_return:+6.2f}%")
    
    print(f"\n✅ Generated data for {len(ipo_data)} IPOs")

elif DATA_MODE == 'IBKR':
    print("🔌 Attempting IBKR connection...\n")
    
    try:
        from ib_insync import IB, Stock, util
        
        ib = IB()
        ib.connect(
            IBKR_CONFIG['HOST'], 
            IBKR_CONFIG['PORT'], 
            clientId=IBKR_CONFIG['CLIENT_ID']
        )
        print(f"✓ Connected to IBKR with client ID {IBKR_CONFIG['CLIENT_ID']}\n")
        
        # Pull data for each ticker
        for _, row in ipo_universe.iterrows():
            ticker = row['Ticker']
            ipo_date = row['IPO_Date']
            
            try:
                contract = Stock(ticker, 'SMART', 'USD')
                qualified = ib.qualifyContracts(contract)
                
                if qualified:
                    eastern = pytz.timezone('America/New_York')
                    end_dt = eastern.localize(ipo_date.replace(hour=23, minute=59))
                    end_str = end_dt.strftime("%Y%m%d %H:%M:%S") + " US/Eastern"
                    
                    bars = ib.reqHistoricalData(
                        qualified[0],
                        endDateTime=end_str,
                        durationStr='1 D',
                        barSizeSetting='1 min',
                        whatToShow='TRADES',
                        useRTH=False,
                        formatDate=1
                    )
                    
                    if bars:
                        df = util.df(bars)
                        df['datetime'] = pd.to_datetime(df['date'])
                        df = df.drop('date', axis=1)
                        ipo_data[ticker] = df
                        print(f"✓ {ticker}: Retrieved {len(df)} bars")
                    else:
                        print(f"⚠️ {ticker}: No data available")
                else:
                    print(f"⚠️ {ticker}: Could not qualify contract")
                    
                time.sleep(IBKR_CONFIG['REQUEST_DELAY'])
                
            except Exception as e:
                print(f"❌ {ticker}: {str(e)[:50]}")
        
        ib.disconnect()
        print(f"\n✓ Retrieved data for {len(ipo_data)}/{len(ipo_universe)} tickers")
        
    except Exception as e:
        print(f"❌ IBKR connection failed: {e}")
        print("\n🔄 Falling back to simulated data...\n")
        
        # Fallback to simulation
        for _, row in ipo_universe.iterrows():
            ticker = row['Ticker']
            ipo_date = row['IPO_Date']
            ipo_price = row['IPO_Price']
            sector = row.get('Sector', 'Tech')
            
            df = generate_ipo_day_data(ticker, ipo_date, ipo_price, sector)
            ipo_data[ticker] = df
            
            open_price = df.iloc[0]['open']
            close_price = df.iloc[-1]['close']
            day_return = (close_price - open_price) / open_price * 100
            
            print(f"✓ {ticker:6s} | Open: ${open_price:6.2f} | Close: ${close_price:6.2f} | Return: {day_return:+6.2f}%")
        
        print(f"\n✅ Generated fallback data for {len(ipo_data)} IPOs")

print(f"\n📊 Data ready for analysis!")

## 5️⃣ Window Analysis Engine

In [ ]:
def analyze_trading_windows(data_dict):
    """Comprehensive analysis of all trading window combinations"""
    
    # Generate all possible windows (30-minute intervals)
    times = []
    for hour in range(9, 16):
        for minute in [0, 30]:
            if hour == 9 and minute == 0:
                continue  # Skip 9:00
            if hour == 16 and minute == 30:
                continue  # Market closes at 4:00
            times.append(f"{hour:02d}:{minute:02d}")
    
    results = []
    
    # Test all window combinations
    for i, buy_time_str in enumerate(times[:-1]):
        for sell_time_str in times[i+1:]:
            buy_time = pd.to_datetime(buy_time_str).time()
            sell_time = pd.to_datetime(sell_time_str).time()
            
            returns = []
            
            for ticker, df in data_dict.items():
                # Find prices at specified times
                buy_mask = df['datetime'].dt.time == buy_time
                sell_mask = df['datetime'].dt.time == sell_time
                
                if buy_mask.any() and sell_mask.any():
                    buy_price = df.loc[buy_mask, 'close'].iloc[0]
                    sell_price = df.loc[sell_mask, 'close'].iloc[0]
                    
                    if buy_price > 0:
                        ret = (sell_price - buy_price) / buy_price
                        returns.append(ret)
            
            if len(returns) >= 5:  # Minimum sample size
                duration = (datetime.strptime(sell_time_str, '%H:%M') - 
                          datetime.strptime(buy_time_str, '%H:%M')).seconds / 3600
                
                results.append({
                    'window': f"{buy_time_str}-{sell_time_str}",
                    'buy_time': buy_time_str,
                    'sell_time': sell_time_str,
                    'duration_hrs': duration,
                    'n': len(returns),
                    'avg_return': np.mean(returns) * 100,
                    'median_return': np.median(returns) * 100,
                    'std_return': np.std(returns) * 100,
                    'win_rate': sum(1 for r in returns if r > 0) / len(returns) * 100,
                    'sharpe': np.mean(returns) / np.std(returns) * np.sqrt(252) if np.std(returns) > 0 else 0,
                    'max_return': max(returns) * 100,
                    'min_return': min(returns) * 100,
                    'return_per_hour': (np.mean(returns) * 100) / duration if duration > 0 else 0
                })
    
    return pd.DataFrame(results).sort_values('avg_return', ascending=False).reset_index(drop=True)

# Run analysis
print("🔍 Analyzing trading windows...\n")
results_df = analyze_trading_windows(ipo_data)

# Save results
results_df.to_csv('outputs/window_analysis_results.csv', index=False)
print(f"✓ Analyzed {len(results_df)} trading windows")
print(f"✓ Saved results to outputs/window_analysis_results.csv\n")

# Display top performers
print("🏆 TOP 10 PERFORMING WINDOWS:")
print("="*70)

for idx, row in results_df.head(10).iterrows():
    print(f"{idx+1:2d}. {row['window']:11s} | "
          f"Return: {row['avg_return']:+6.2f}% | "
          f"Win: {row['win_rate']:5.1f}% | "
          f"Sharpe: {row['sharpe']:+6.2f} | "
          f"n={int(row['n'])}")

print("\n📊 Quick Stats:")
print(f"   Profitable windows: {sum(results_df['avg_return'] > 0)}/{len(results_df)}")
print(f"   High win rate (>60%): {sum(results_df['win_rate'] > 60)} windows")
print(f"   Best return: {results_df['avg_return'].max():.2f}%")
print(f"   Best Sharpe: {results_df['sharpe'].max():.2f}")

## 6️⃣ Advanced Visualizations

In [ ]:
# Create comprehensive visualization dashboard
plt.style.use('seaborn-v0_8-whitegrid')
fig = plt.figure(figsize=(20, 12))
gs = GridSpec(3, 4, hspace=0.3, wspace=0.3, figure=fig)

# Color palette
colors = sns.color_palette("husl", 10)

# 1. Top Windows Bar Chart
ax1 = fig.add_subplot(gs[0, :2])
top15 = results_df.head(15)
bar_colors = ['green' if x > 0 else 'red' for x in top15['avg_return']]
bars = ax1.bar(range(len(top15)), top15['avg_return'], color=bar_colors, alpha=0.7)
ax1.set_xticks(range(len(top15)))
ax1.set_xticklabels(top15['window'], rotation=45, ha='right', fontsize=9)
ax1.set_ylabel('Average Return (%)', fontsize=10)
ax1.set_title('Top 15 Trading Windows by Average Return', fontsize=12, fontweight='bold')
ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax1.grid(True, alpha=0.3)

# Add value labels
for bar, val in zip(bars, top15['avg_return']):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.1f}%', ha='center', va='bottom' if val > 0 else 'top', fontsize=8)

# 2. Risk vs Return Scatter
ax2 = fig.add_subplot(gs[0, 2])
scatter = ax2.scatter(results_df['std_return'], results_df['avg_return'],
                     s=results_df['n']*3, alpha=0.6,
                     c=results_df['sharpe'], cmap='RdYlGn', vmin=-1, vmax=1)
ax2.set_xlabel('Risk (Std Dev %)', fontsize=10)
ax2.set_ylabel('Return (%)', fontsize=10)
ax2.set_title('Risk-Return Profile', fontsize=12, fontweight='bold')
ax2.axhline(y=0, color='black', linestyle='--', alpha=0.3)
ax2.axvline(x=results_df['std_return'].median(), color='blue', linestyle='--', alpha=0.3)
plt.colorbar(scatter, ax=ax2, label='Sharpe Ratio')
ax2.grid(True, alpha=0.3)

# 3. Win Rate Distribution
ax3 = fig.add_subplot(gs[0, 3])
ax3.hist(results_df['win_rate'], bins=20, color='steelblue', alpha=0.7, edgecolor='black')
ax3.axvline(x=50, color='red', linestyle='--', linewidth=2, label='50% Line')
ax3.set_xlabel('Win Rate (%)', fontsize=10)
ax3.set_ylabel('Frequency', fontsize=10)
ax3.set_title('Win Rate Distribution', fontsize=12, fontweight='bold')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Returns Heatmap
ax4 = fig.add_subplot(gs[1, :2])
pivot_returns = results_df.pivot_table(
    values='avg_return',
    index='buy_time',
    columns='sell_time',
    aggfunc='first'
)
sns.heatmap(pivot_returns, annot=True, fmt='.1f', cmap='RdYlGn',
           center=0, ax=ax4, cbar_kws={'label': 'Avg Return (%)'},
           linewidths=0.5, linecolor='gray')
ax4.set_title('Trading Windows Heatmap - Returns (%)', fontsize=12, fontweight='bold')
ax4.set_xlabel('Sell Time', fontsize=10)
ax4.set_ylabel('Buy Time', fontsize=10)

# 5. Win Rate Heatmap
ax5 = fig.add_subplot(gs[1, 2:])
pivot_winrate = results_df.pivot_table(
    values='win_rate',
    index='buy_time',
    columns='sell_time',
    aggfunc='first'
)
sns.heatmap(pivot_winrate, annot=True, fmt='.0f', cmap='YlOrRd',
           vmin=0, vmax=100, ax=ax5, cbar_kws={'label': 'Win Rate (%)'},
           linewidths=0.5, linecolor='gray')
ax5.set_title('Trading Windows Heatmap - Win Rate (%)', fontsize=12, fontweight='bold')
ax5.set_xlabel('Sell Time', fontsize=10)
ax5.set_ylabel('Buy Time', fontsize=10)

# 6. Entry Hour Analysis
ax6 = fig.add_subplot(gs[2, 0])
hour_returns = results_df.copy()
hour_returns['buy_hour'] = pd.to_datetime(hour_returns['buy_time']).dt.hour
hour_avg = hour_returns.groupby('buy_hour')['avg_return'].mean()
ax6.bar(hour_avg.index, hour_avg.values, color='purple', alpha=0.7)
ax6.set_xlabel('Entry Hour', fontsize=10)
ax6.set_ylabel('Average Return (%)', fontsize=10)
ax6.set_title('Returns by Entry Hour', fontsize=12, fontweight='bold')
ax6.set_xticks(range(9, 16))
ax6.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax6.grid(True, alpha=0.3)

# 7. Hold Duration Analysis
ax7 = fig.add_subplot(gs[2, 1])
duration_returns = results_df.groupby('duration_hrs')['avg_return'].mean()
ax7.bar(duration_returns.index, duration_returns.values, color='orange', alpha=0.7)
ax7.set_xlabel('Hold Duration (hours)', fontsize=10)
ax7.set_ylabel('Average Return (%)', fontsize=10)
ax7.set_title('Returns by Hold Duration', fontsize=12, fontweight='bold')
ax7.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax7.grid(True, alpha=0.3)

# 8. Top Windows Summary Table
ax8 = fig.add_subplot(gs[2, 2:])
ax8.axis('tight')
ax8.axis('off')

# Create table data
table_data = []
for idx, row in results_df.head(10).iterrows():
    table_data.append([
        f"{idx+1}",
        row['window'],
        f"{row['avg_return']:+.2f}%",
        f"{row['win_rate']:.0f}%",
        f"{row['sharpe']:+.2f}",
        f"{row['return_per_hour']:+.2f}%/hr",
        f"{int(row['n'])}"
    ])

table = ax8.table(cellText=table_data,
                 colLabels=['Rank', 'Window', 'Avg Return', 'Win Rate', 'Sharpe', 'Return/Hour', 'n'],
                 cellLoc='center',
                 loc='center')
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.8)

# Color code returns
for i in range(len(table_data)):
    return_val = float(table_data[i][2].replace('%', '').replace('+', ''))
    if return_val > 0:
        table[(i+1, 2)].set_facecolor('#90EE90')
    else:
        table[(i+1, 2)].set_facecolor('#FFB6C1')

ax8.set_title('Top 10 Trading Windows - Detailed Metrics', fontsize=12, fontweight='bold', pad=20)

plt.suptitle('IPO Day Trading Strategy - Comprehensive Analysis Dashboard',
             fontsize=16, fontweight='bold', y=1.02)

plt.savefig('outputs/ipo_strategy_dashboard.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Dashboard saved to outputs/ipo_strategy_dashboard.png")

## 7️⃣ Strategy Report Generation

In [ ]:
# Generate comprehensive strategy report
report = []
report.append("="*80)
report.append(" "*25 + "IPO FARMING STRATEGY REPORT")
report.append(" "*28 + datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
report.append("="*80)

report.append("\n📊 EXECUTIVE SUMMARY")
report.append("-"*40)
report.append(f"• Analyzed: {len(results_df)} trading windows")
report.append(f"• IPOs tested: {len(ipo_data)} companies")
report.append(f"• Data mode: {DATA_MODE}")
report.append(f"• Best window: {results_df.iloc[0]['window']}")
report.append(f"• Best return: {results_df.iloc[0]['avg_return']:.2f}%")
report.append(f"• Highest win rate: {results_df['win_rate'].max():.1f}%")
report.append(f"• Best Sharpe ratio: {results_df['sharpe'].max():.2f}")

report.append("\n🏆 TOP 5 STRATEGIES")
report.append("-"*40)
for idx, row in results_df.head(5).iterrows():
    report.append(f"\n{idx+1}. Window: {row['window']}")
    report.append(f"   • Average Return: {row['avg_return']:+.2f}%")
    report.append(f"   • Win Rate: {row['win_rate']:.1f}%")
    report.append(f"   • Sharpe Ratio: {row['sharpe']:+.2f}")
    report.append(f"   • Return per Hour: {row['return_per_hour']:+.2f}%")
    report.append(f"   • Hold Duration: {row['duration_hrs']:.1f} hours")

report.append("\n💡 KEY INSIGHTS")
report.append("-"*40)

# Time-based analysis
morning = results_df[results_df['buy_time'].apply(lambda x: int(x.split(':')[0]) < 12)]
afternoon = results_df[results_df['buy_time'].apply(lambda x: int(x.split(':')[0]) >= 12)]
report.append(f"• Morning entries (before 12:00): Avg return = {morning['avg_return'].mean():.2f}%")
report.append(f"• Afternoon entries (after 12:00): Avg return = {afternoon['avg_return'].mean():.2f}%")

# Duration analysis
short = results_df[results_df['duration_hrs'] <= 1]
medium = results_df[(results_df['duration_hrs'] > 1) & (results_df['duration_hrs'] <= 3)]
long = results_df[results_df['duration_hrs'] > 3]
report.append(f"\n• Short holds (≤1 hour): Avg return = {short['avg_return'].mean():.2f}%")
report.append(f"• Medium holds (1-3 hours): Avg return = {medium['avg_return'].mean():.2f}%")
report.append(f"• Long holds (>3 hours): Avg return = {long['avg_return'].mean():.2f}%")

# Win rate analysis
high_wr = results_df[results_df['win_rate'] > 60]
report.append(f"\n• High win rate strategies (>60%): {len(high_wr)} windows")
report.append(f"• Their average return: {high_wr['avg_return'].mean():.2f}%")
report.append(f"• Their average Sharpe: {high_wr['sharpe'].mean():.2f}")

report.append("\n📈 OPTIMAL STRATEGY")
report.append("-"*40)

# Find optimal strategy (balance of return, win rate, and Sharpe)
results_df['score'] = (
    results_df['avg_return'] * 0.4 + 
    results_df['win_rate'] * 0.3 + 
    results_df['sharpe'] * 0.3
)
optimal = results_df.nlargest(1, 'score').iloc[0]

report.append(f"Recommended Window: {optimal['window']}")
report.append(f"  • Enter at: {optimal['buy_time']}")
report.append(f"  • Exit at: {optimal['sell_time']}")
report.append(f"  • Expected Return: {optimal['avg_return']:.2f}%")
report.append(f"  • Win Rate: {optimal['win_rate']:.1f}%")
report.append(f"  • Risk-Adjusted Return: {optimal['sharpe']:.2f} Sharpe")

report.append("\n⚠️ RISK CONSIDERATIONS")
report.append("-"*40)
report.append(f"• Average volatility: {results_df['std_return'].mean():.2f}%")
report.append(f"• Worst drawdown observed: {results_df['min_return'].min():.2f}%")
report.append(f"• Windows with negative returns: {sum(results_df['avg_return'] < 0)}/{len(results_df)}")
report.append(f"• Recommended stop-loss: -5% from entry")
report.append(f"• Position sizing: Risk 1-2% of capital per trade")

report.append("\n🎯 IMPLEMENTATION GUIDELINES")
report.append("-"*40)
report.append("1. Pre-market preparation:")
report.append("   • Identify IPOs scheduled for the day")
report.append("   • Check for expected opening price range")
report.append("   • Set alerts for entry time")
report.append("\n2. Entry criteria:")
report.append("   • Wait for initial volatility to settle (10:00-10:30 AM ideal)")
report.append("   • Confirm positive momentum before entry")
report.append("   • Use limit orders to control entry price")
report.append("\n3. Exit strategy:")
report.append("   • Set target based on selected window")
report.append("   • Use trailing stop for winners")
report.append("   • Exit before 3:30 PM to avoid closing volatility")

report.append("\n" + "="*80)
report.append("END OF REPORT")
report.append("="*80)

# Save report
report_text = '\n'.join(report)
with open('outputs/ipo_strategy_report.txt', 'w') as f:
    f.write(report_text)

print(report_text)
print("\n✅ Report saved to outputs/ipo_strategy_report.txt")

## 8️⃣ Export Results & Summary

In [ ]:
# Create final summary
print("="*80)
print(" "*30 + "✅ ANALYSIS COMPLETE!")
print("="*80)

print("\n📁 FILES GENERATED:")
print("-"*40)

output_files = [
    ('outputs/window_analysis_results.csv', 'Complete window analysis data'),
    ('outputs/ipo_strategy_dashboard.png', 'Comprehensive visual dashboard'),
    ('outputs/ipo_strategy_report.txt', 'Detailed strategy report')
]

for file, description in output_files:
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024  # KB
        print(f"✓ {file:40s} ({size:6.1f} KB) - {description}")
    else:
        print(f"⚠ {file:40s} - Not found")

print("\n🎯 QUICK ACTION ITEMS:")
print("-"*40)
print("1. Review the dashboard visualization for patterns")
print("2. Read the strategy report for detailed insights")
print("3. Test top 3 windows with paper trading")
print("4. Adjust parameters based on your risk tolerance")
print("5. Consider market conditions before live trading")

print("\n💰 TOP 3 RECOMMENDED WINDOWS:")
print("-"*40)

for idx, row in results_df.head(3).iterrows():
    print(f"\n{idx+1}. {row['window']}")
    print(f"   Enter: {row['buy_time']} | Exit: {row['sell_time']}")
    print(f"   Expected: {row['avg_return']:+.1f}% return with {row['win_rate']:.0f}% win rate")

print("\n" + "="*80)
print("📈 Happy Trading! Remember to always manage your risk properly.")
print("="*80)

## 🎉 Congratulations!

You've successfully completed the IPO Farming Backtest analysis!

### 📋 What You've Accomplished:
- ✅ Analyzed multiple IPO trading windows
- ✅ Identified optimal entry and exit times
- ✅ Generated comprehensive visualizations
- ✅ Created actionable strategy report

### 🚀 Next Steps:
1. **Paper Trade**: Test the top strategies with simulated money
2. **Refine**: Adjust parameters based on your observations
3. **Scale**: Gradually increase position sizes as confidence builds
4. **Monitor**: Keep track of IPO calendars for opportunities

### ⚠️ Remember:
- Past performance doesn't guarantee future results
- Always use proper risk management
- Start small and scale gradually
- Market conditions change - adapt accordingly

### 📞 Support:
- Modify `DATA_MODE` to switch between simulation and live IBKR data
- Adjust `MAX_TICKERS` to analyze more or fewer IPOs
- Change analysis parameters in the cells above as needed

**Good luck with your IPO trading strategy! 🍀**